In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [3]:
import os, sys 
%cd ~/nbs/
sys.path.append(os.path.join(os.getcwd(), "orig" ))

%matplotlib inline 
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

/home/ubuntu/nbs


Using Theano backend.


1. Create batches of validation 
2. Create batches of training 
3. Use average of different models and fit it collectively 
4. Fit the batches
5. Run the training on the train and validation data. 
6. Predict on the test 
7. Run prediction on validation set 
8. See what of the predicted on the validation set is correct 
9. Submit tests to backend 

In [8]:
path = 'data/dogscats/'
model_path = path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

In [10]:
??get_classes
training_batches = get_batches(path + 'train', shuffle=False, batch_size=batch_size)
validation_batches = get_batches(path + 'valid', shuffle=False, batch_size=batch_size)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [11]:
(val_classes, training_classes, val_labels, training_lables, 
val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


# Create an ensemble of different models

1. Train model with last layer all other layer traininable is false
2. Add model with everything but the last layer. This creates an ensemble. 
3. Fine tune just the dense layers of this model. 
4. Add data augmentation, fine tuning dense layers without computation

In [23]:
model = Vgg16().model
print(model.summary())
conv_layers, fc_layers = split_at(model, Convolution2D)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_11 (Lambda)               (None, 3, 224, 224)   0           lambda_input_11[0][0]            
____________________________________________________________________________________________________
zeropadding2d_131 (ZeroPadding2D (None, 3, 226, 226)   0           lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution2d_131 (Convolution2D (None, 64, 224, 224)  1792        zeropadding2d_131[0][0]          
____________________________________________________________________________________________________
zeropadding2d_132 (ZeroPadding2D (None, 64, 226, 226)  0           convolution2d_131[0][0]          
___________________________________________________________________________________________

In [25]:
conv_model = Sequential(conv_layers)
print(conv_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_11 (Lambda)               (None, 3, 224, 224)   0           lambda_input_11[0][0]            
____________________________________________________________________________________________________
zeropadding2d_131 (ZeroPadding2D (None, 3, 226, 226)   0           lambda_11[0][0]                  
                                                                   lambda_11[0][0]                  
                                                                   lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution2d_131 (Convolution2D (None, 64, 224, 224)  1792        zeropadding2d_131[0][0]          
                                                                   zeropadding2d_131[1][0] 

convolution2d_140 (Convolution2D (None, 512, 28, 28)   2359808     zeropadding2d_140[0][0]          
                                                                   zeropadding2d_140[1][0]          
                                                                   zeropadding2d_140[2][0]          
____________________________________________________________________________________________________
maxpooling2d_54 (MaxPooling2D)   (None, 512, 14, 14)   0           convolution2d_140[0][0]          
                                                                   convolution2d_140[1][0]          
                                                                   convolution2d_140[2][0]          
____________________________________________________________________________________________________
zeropadding2d_141 (ZeroPadding2D (None, 512, 16, 16)   0           maxpooling2d_54[0][0]            
                                                                   maxpooling2d_54[1][0]   

In [27]:
conv_model.predict_generator(validation_batches, validation_batches.nb_sample)

array([[[[  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         ..., 
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   1.6217e+01]],

        [[  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
         ..., 
         [  0.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+0